## Imports

In [10]:
import os
import cv2
import json
import pandas as pd

## Step 1: Load the video

In [8]:
def load_video(video_name, folder_path="../../data/train_sample_videos"):
    video_path = os.path.join(folder_path, video_name)
    if not os.path.exists(video_path):
        raise FileNotFoundError(f"Video {video_name} not found in {folder_path}")

    cap = cv2.VideoCapture(video_path)
    if not cap.isOpened():
        raise ValueError(f"Could not open the video: {video_name}")

    print(f"Successfully loaded video: {video_name}")
    return cap

In [9]:
# Testing the function
video_name = "aagfhgtpmv.mp4"  # Replace with an actual video filename in your folder
cap = load_video(video_name)

Successfully loaded video: aagfhgtpmv.mp4
